In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

In [18]:
llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo"
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the following context and chat history to answer the user's question. "
               "If you don't know the answer, say you don't know.\n\nContext:\n{context}\n\nChat history:\n{chat_history}"),
    ("human", "{question}")
])

In [19]:
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [20]:
chain = {
    "context": retriever,
    "question": RunnablePassthrough(),
    "chat_history": lambda x: memory.load_memory_variables({})["chat_history"]
} | prompt | llm

In [21]:
def ask(question: str):
    result = chain.invoke(question)
    memory.save_context({"question": question}, {"output": result.content})
    return print(result.content)

In [22]:
ask("Is Aaronson guilty?")

In the text, it is mentioned that Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with. However, it is also stated that the protagonist had never seen the photograph that disproved their guilt and that it had never existed. This ambiguity suggests that the guilt of Aaronson and the others may not be as clear-cut as it seems.


In [23]:
ask("What message did he write in the table?")

In the text, Winston traced with his finger in the dust on the table the message "2+2=5."


In [24]:
ask("Who is Julia?")

Julia is a character in the novel who plays a significant role in Winston's life. She is a woman whom Winston falls in love with and has a secret relationship with. Julia is portrayed as a rebellious and passionate individual who shares Winston's disdain for the Party and their oppressive regime. Their relationship becomes a symbol of resistance against the totalitarian rule of the Party in the novel.


In [25]:
ask("What was the first question I asked?")

The first question you asked was, "Is Aaronson guilty?"
